Notebook em complemento aos meus outros trabalhos, todos então disponíveis em meu github. Esse é um dos desafios disponíveis no Kaggle sobre o covid-19, e tentarei durante o trabalho realizar algumas das taferas.

https://github.com/LeopoldoZanellato/python

https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/tasks


O dataset disponível hoje, tem um total de mais de 13GB e 120 mil artigos, no que eu trabalharei possui cerca de 3GB e 20 mil artigos.
Um dos motivos pelo qual não utilizarei o de 13GB é de que utilizaria muita memória e tomaria muito tempo, e esse não é o propósito. O propósito é de apresentar pequenas soluções e demonstrar como o uso do machine learning poderá ajudar os profissionais de saúdes no combate ao covid-19

Esse trabalho será divido em 2 partes:

Primeira parte a de exploração de dados dos artigos científicos, onde buscarei algumas respostas pelos desafios e como de uma maneira facil poderemos responder todos eles

Segunda parte é a simulação de predições com a base de dados disponível pelo ministério da saúde. 


Começaremos pelo tratamento dos dados:

In [ ]:
!pip install scispacy

In [ ]:
!pip install pmdarima

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_md-0.2.4.tar.gz

In [ ]:
!python -m spacy download en

In [ ]:
import glob, json, zipfile, en_core_sci_md
"""from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, CustomJS
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap
from bokeh.io import output_file, show
from bokeh.transform import transform
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import RadioButtonGroup
from bokeh.models import TextInput
from bokeh.layouts import gridplot
from bokeh.models import Div
from bokeh.models import Paragraph
from bokeh.layouts import column, widgetbox"""

import numpy as np
import pandas as pd
import seaborn as sns
import spacy, scispacy, operator
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.display import HTML
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from statsmodels.tools.eval_measures import rmse, mse
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.statespace.tools import diff
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
"""from google.colab import drive
drive.mount("/content/gdrive")""";

In [ ]:
"""import zipfile
path = '/content/gdrive/My Drive/CORD-19-research-challenge.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()""";

In [ ]:
corona_features = {'paper_id': [], 'title': [],
                   'abstract': [], 'text': []}

In [ ]:
corona_df = pd.DataFrame.from_dict(corona_features)

In [ ]:
json_filenames = glob.glob(f'{"./"}//**/*.json', recursive = True)

In [ ]:
def return_corona_df(json_filenames, df):
    "Função para ler os arquivos json"
    for file_name in json_filenames:
        row = {'paper_id': None, 'title': None,
           'abstract': None,'text': None}
    
        with open(file_name) as json_data:
            try:
                if file_name == "./sample_data/anscombe.json":
                    continue
      
                data = json.load(json_data)

                row['paper_id'] = data['paper_id'].strip()
                row['title'] = data['metadata']['title'].strip()

                abstract_list = [abstract['text'] for abstract in data['abstract']]
                abstract = '\n '.join (abstract_list)
                row['abstract'] = abstract.strip()

                text_list = [text['text'] for text in data['body_text']]
                text = '\n '.join(text_list)
                row['text'] = text.strip()

                df = df.append(row, ignore_index = True)
            except:
                pass 
    return df


In [ ]:
corona_df = return_corona_df(json_filenames, corona_df)

In [ ]:
corona_df.head()

In [ ]:
corona_df = corona_df[corona_df['title']!= ""]
corona_df = corona_df[corona_df['abstract']!= ""]

In [ ]:
corona_df.drop_duplicates(['abstract','text','title'], inplace=True)

In [ ]:
corona_df.shape

In [ ]:
corona_df.to_csv("pre_processado.csv")

In [ ]:
nlp = en_core_sci_md.load(disable = ['tagger', 'parser', 'ner'])
nlp.max_length = 2000000

In [ ]:
new_stop_words = ['et', 'al','doi','cppyright','http', 
                  'https', 'fig','table','result','show']
for word in new_stop_words:
    nlp.vocab[word].is_stop = True

Durante o tratamento dos dados realizaremos a lemmatização, e a função abaixo servirá para isso.


A lematização é uma técnica, geralmente utilizada por buscador de palavras em sites, para abranger a quantidade de opções de palavras relacionadas a palavra buscada, ignorando o tempo verbal caso seja um verbo, o gênero da palavra, o plural e etc.

Por exemplo, as palavras pensamento, pensando, e derivadas delas serão trocadas todas por "pensar", facilitando assim a "padronização" do texto.

Serão retirados também todos os números (que estão sozinhos), stop words (palavras de ligação que são irrelevantes ao conjunto dos resultados), pontuações e espaços indevidos 

In [ ]:
def spacy_tokenizer(sentence):
    sentence = sentence.lower()
    lista = []
    lista = [word.lemma_ for word in nlp(sentence) if not (word.is_stop or
                                                         word.like_num or
                                                         word.is_punct or 
                                                         word.is_space or
                                                         len(word)==1)]
    lista = ' '.join([str(element) for element in lista])
    return lista

In [ ]:
corona_df['text'] = corona_df['text'].apply(spacy_tokenizer)

In [ ]:
corona_df.to_csv("pos_processamento.csv", encoding = "UTF-8")

In [ ]:
corona_df_completo = pd.read_csv("../input/coronavirus/pos_processamento.csv", index_col = 0)
#corona_df_completo = pd.read_csv("../input/CORD-19-research-challenge/metadata.csv", index_col=0)

In [ ]:
corona_df_completo.head()

Após o tratamento de dados podemos ver como ficou um dos textos

In [ ]:
print(corona_df_completo.iloc[0]["text"])

In [ ]:
corona_df_completo.dropna(inplace=True)

In [ ]:
corona_df_completo.shape

In [ ]:
corona_df_completo.head()

Aqui estão todos os artigos, cada um com o seu respectivo ID, título , abstract e o texto

In [ ]:
dataset_texts = corona_df_completo['text'].tolist()

In [ ]:
len(dataset_texts)

In [ ]:
tfidf = TfidfVectorizer(max_features=2**12) #utilização desse vetor para limitação do tamanho da matriz esparsa
vectorized = tfidf.fit_transform(dataset_texts)
vectorized

Redução de dimensionalidade para separação dos artigos em classes:

In [ ]:

pca = PCA(n_components = 2)
X_pca = pca.fit_transform(vectorized.toarray())

In [ ]:
components = pca.explained_variance_ratio_
components

A seguir será feita a definição dos clusters, apesar do gráfico demonstar que seriam interessantes 15 clusters, apenas para fins de demonstração, farei apenas com 5 grupos para uma melhor visualização, o interessante que conforme passamos o mouse pelos pontos, podemos então ler cada um dos títulos e o contexto do artigo

In [ ]:
wcss = []
for i in range(1,21):
    kmeans = MiniBatchKMeans(n_clusters = i, random_state = 0)
    kmeans.fit(vectorized)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,21), wcss)
plt.xlabel('Number of Clusters')
plt.ylabel("WCSS")

In [ ]:
k = 5
kmeans = MiniBatchKMeans(n_clusters=k, random_state = 16)
y_pred = kmeans.fit_predict(vectorized)

In [ ]:
np.unique(y_pred)

In [ ]:
# Baseado em: https://www.kaggle.com/maksimeren/covid-19-literature-clustering


output_notebook()
y_labels = y_pred

# data sources
source = ColumnDataSource(data=dict(
    x= X_pca[:,0], 
    y= X_pca[:,1],
    x_backup = X_pca[:,0],
    y_backup = X_pca[:,1],
    desc= y_labels, 
    titles= corona_df_completo['title'],
    abstract = corona_df_completo['abstract'],
    labels = ["C-" + str(x) for x in y_labels]
    ))

# hover over information
hover = HoverTool(tooltips=[
    ("Title", "@titles{safe}"),
    ("Abstract", "@abstract{safe}"),
],
                 point_policy="follow_mouse")

# map colors
mapper = linear_cmap(field_name='desc', 
                     palette=Category20[9],
                     low=min(y_labels) ,high=max(y_labels))

# prepare the figure
p = figure(plot_width=800, plot_height=800, 
           tools=[hover, 'pan', 'wheel_zoom', 'box_zoom', 'reset'], 
           title="Covid-19 Papers", 
           toolbar_location="right")

# plot
p.scatter('x', 'y', size=5, 
          source=source,
          fill_color=mapper,
          line_alpha=0.3,
          line_color="black",
          legend = 'labels')

# add callback to control 
callback = CustomJS(args=dict(p=p, source=source), code="""
            
            var radio_value = cb_obj.active;
            var data = source.data; 
            
            x = data['x'];
            y = data['y'];
            
            x_backup = data['x_backup'];
            y_backup = data['y_backup'];
            
            labels = data['desc'];
            
            if (radio_value == '5') {
                for (i = 0; i < x.length; i++) {
                    x[i] = x_backup[i];
                    y[i] = y_backup[i];
                }
            }
            else {
                for (i = 0; i < x.length; i++) {
                    if(labels[i] == radio_value) {
                        x[i] = x_backup[i];
                        y[i] = y_backup[i];
                    } else {
                        x[i] = undefined;
                        y[i] = undefined;
                    }
                }
            }


        source.change.emit();
        """)


# option
option = RadioButtonGroup(labels=["C-0", "C-1", "C-2",
                                  "C-3", "C-5",
                                   "All"], 
                          active=5, callback=callback)

#header
header = Div(text="""<h1>Covid-19 Papers</h1>""")

# show
show(column(header, widgetbox(option),p))

In [ ]:
from IPython.display import Image
Image(filename='../input/coronavirus/imagem3.jpg', width=500, height=500)

Como a função acima trás um gráfico panoramico com algumas funcionalidades, na hora de postar no github deu máximo de tamanho permitido, para solucionar esse problema, tive importar o print da imagem de como o gráfico funciona. 

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(X_pca[:,0], X_pca[:,1], hue=y_pred,palette="bright")
plt.title("Covid-19 Papers")

Até agora foi utilizado o dataset completo, a partir desse ponto, utilizarei apenas uma amostra de 500 artigos para que não demore muito tempo. O propósito é demonstrar as ferramentas e como podem ser utilizadas

In [ ]:
corona_df = corona_df_completo.copy()

In [ ]:
corona_df = corona_df.sample(n = 500, random_state = 0)

In [ ]:
corona_df

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [ ]:
text = str(corona_df['text'][25534][:1000])


In [ ]:
nlp_ent = spacy.load("en_core_web_sm")
nlp_ent.max_length = 2000000

In [ ]:
doc = nlp_ent(text)

Utilizarei a biblioteca Spacy para fazer a leitura de linguagem natural. Trouxe uma definição de linguagem natural segundo o site do wikipedia:

Língua natural (língua humana, língua idiomática, ou somente língua ou idioma) é qualquer linguagem desenvolvida naturalmente pelo ser humano, de forma não premeditada, como resultado da facilidade inata para a linguagem possuída pelo intelecto humano. Vários exemplos podem ser dados como as línguas faladas e as línguas de sinais. A linguagem natural é normalmente utilizada para a comunicação. As línguas naturais são diferentes das línguas construídas e das línguas formais, tais como a linguística computacional, a língua escrita, a linguagem animal e as linguagens usadas no estudo formal da lógica, especialmente da lógica matemática.


Um exemplo disso é que o próprio algoritmo ja define o que é localização, pessoas, nacionalidade, nomes próprio etc... Assim, podemos verificar os países que são mais citados nos textos


Abaixo podemos ver como o modelo se comporta quando separa as entidades, pessoas, datas, localizações etc...

In [ ]:
from spacy import displacy
displacy.render(doc,style = 'ent', jupyter = True)

In [ ]:
gpe = []
for index, row in corona_df.iterrows():
    text = row['text']
    doc = nlp_ent(text)
    for entity in doc.ents:
        if entity.label_ =='GPE':
            gpe.append(str(entity.text))

In [ ]:
values_gpe,counts_gpe = np.unique(np.array(gpe), return_counts = True)

In [ ]:
gpe_df = pd.DataFrame({'value': values_gpe, 'counts': counts_gpe})

In [ ]:
gpe_df = gpe_df.sort_values(by='counts', ascending=False).head(8)

In [ ]:
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = round(float(rect.get_height()),2)
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
rect1 = ax.bar(x=gpe_df["value"], height=gpe_df["counts"])
plt.title("Citações dos países nos 500 textos")
plt.ylabel('Citações de cada País / localização')
plt.xlabel('País/Localização')
autolabel(rect1)

Agora começarei a parte de como explorar os textos!! Faremos a busca por palavras e nos retornará o texto, abstract e o título do texto

In [ ]:
def find_all_texts(input_str, search_str, number_of_words):
    text_list = []
    index = 0
    number_of_words = number_of_words
    while index < len(input_str):
        i = input_str.find(search_str, index)
        if i == -1:
            return text_list
    
        if input_str[i-number_of_words:i] == '':
            start = 0
        else:
            start = i - number_of_words
    
        text_list.append(input_str[start:i] + input_str[i:i + number_of_words])
        index = i + i
    return text_list

In [ ]:
nlp = en_core_sci_md.load(disable = ['tagger', 'parser', 'ner'])
nlp.max_length = 2000000

In [ ]:
def spacy_tokenizer(sentence):
    sentence = sentence.lower()
    lista = []
    lista = [word.lemma_ for word in nlp(sentence) if not (word.is_stop or
                                                         word.like_num or
                                                         word.is_punct or 
                                                         word.is_space or
                                                         len(word)==1)]
    lista = ' '.join([str(element) for element in lista])
    return lista

Até agora foi utilizado a opção de de busca de apenas 1 palavra, onde aparece o Título, ID e quantas vezes a palavra desejada apareceu no artigo, facilitando assim a busca por conteúdo.

A seguir, será feita a busca por mais de uma palavra, conforme o desejo do especialista de saúde

In [ ]:
search_strings = ["traveler"]
tokens_list = [nlp(spacy_tokenizer(item)) for item in search_strings]
tokens_list

In [ ]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)
matcher.add("SEARCH", None, *tokens_list)
number_of_words = 50

In [ ]:
corona_df_200 = corona_df.copy()

In [ ]:
corona_df_200 = corona_df_200[:200]

In [ ]:

for index, row in corona_df_200.iterrows():
    marked_text = ""
    doc = nlp(row["text"])
    paper_id = row["paper_id"]
    title=row['title']
    matches = matcher(doc)
    if matches == []:
        continue
        
    print(f"\n \n \nWords: {search_strings}\n")
    print(f"Title: {title}\n")
    print(f"Paper ID: {paper_id}\n")
    print(f"Matches: {len(matches)}\n")
    
    
    for i in matches:
        start=i[1] - number_of_words
        if start<0:
            start=0
        for j in range(len(tokens_list)):
            if doc[i[1]:i[2]].similarity(tokens_list[j]) ==1:
                search_text = str(tokens_list[j])
                market_text = str(doc[start:i[2] + number_of_words]).replace(search_text, search_text)
                print(f"TEXTO: {market_text}")


O próximo desafio é verificar a similaridade de alguns textos com os artigos, obtendo aqueles artigos que possuem uma maior similaridade

Um dos textos propostos é: 'Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases. Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments'

In [ ]:
texts = corona_df['text'].tolist()

In [ ]:
tfidf = TfidfVectorizer()
vectorized = tfidf.fit_transform(texts)

In [ ]:
search_string = 'Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases. Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments'

In [ ]:
search_string = spacy_tokenizer(search_string)
print(search_string)

In [ ]:
search_string_vectorized = tfidf.transform([search_string])

In [ ]:
similarity = cosine_similarity(search_string_vectorized, vectorized)
similarity

In [ ]:
scores_dict = {}
for i in range(len(similarity[0])):
    scores_dict[i] = similarity[0][i]

In [ ]:
sorted_scores = sorted(scores_dict.items(), key=operator.itemgetter(1), reverse = True)

Aqui então temos por ordem de importância, aqueles artigos onde foi encontrado uma maior similaridade com a frase pesquisada, lembrando que aqui estamos utilizando apenas em 500 artigos, porém pode ser utilizado em todos os outros do banco de dados

In [ ]:
for i in sorted_scores[:5]:
    df=corona_df.iloc[i[0]]
    print(f"Title: {df['title']}")
    print(f"Paper ID: {df['paper_id']}")
    print(f"Score: {i[1]}")
    print(f"Abstract: {str(df['abstract'])[0:500]}")
    print("--------------------------------------------------------------------------------------------------------------------------------------")

A seguir será realizado alguns testes para saber se é possível prever com algum modelo o número de mortes pelo covid aqui no Brasil, e sempre lembrando que esse trabalho é apenas para fins teóricos e de aprendizado. Explicarei também um pouco do modelo que será utilizado para as previsões. A intenção não é contar mortes!!
O primeiro dataset que estarei trabalhando, a data irá do dia 17/03 (dia do primeiro caso de morte) até o dia 28/6, e então farei previsões utilizando modelos estatísticos e de machine learning com a finalidade de tentar prever os próximos dias

In [ ]:
df_brasil = pd.read_csv("../input/coronavirus/brazil_covid19_macro.csv")

In [ ]:
brasil = df_brasil[['date', 'deaths']].groupby('date').sum().reset_index()
brasil = brasil[brasil['deaths'] >0]
brasil['date'] = pd.to_datetime(brasil['date'])
brasil.set_index('date', inplace=True)
brasil.index.freq = "D"

In [ ]:
brasil

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Covid-19 no Brasil")
plt.plot(brasil.values,label="nº Mortes")
plt.xlabel("Dias")
plt.legend()

In [ ]:
plot_acf(brasil['deaths'], lags=40);

In [ ]:
plot_pacf(brasil['deaths'], lags=40);

In [ ]:
seasonal = seasonal_decompose(brasil['deaths'], model='aditive');
seasonal.plot();

Podemos então dividir o primeiro gráfico entre a tendência, a sazonalidade e o residuo

A tendência é em que forma que o gráfico está se aprensentando? Em alta? Em queda?

Sazonalidade é os períodos que se repetem, por exemplo, no inverno as pessoas costumam viajar mais do que no verão, e isso se repete todo ano

Residuo é tudo aquilo que não pode ser explicado pela tendência e pela sazonalidade

A seguir podemos aproximar a sazonalidade e afirmar que existe uma sazonalidade semanal aos finais de semana, isso poderia ser explicado pelo atraso dos dados aos sábados e domingos

In [ ]:
plt.figure(figsize=(12,5))
seasonal.seasonal.plot();
plt.title("Seasonal")
plt.xlabel('Date')

Abaixo temos o gráfico de tendência, como ja poderíamos supor, existe uma tendência de alta nos números de mortes

In [ ]:
plt.figure(figsize=(12,5))
seasonal.trend.plot()
plt.title("Tendência")
plt.xlabel('Date')

Abaixo vemos o gráfico de resíduos, que é tudo aquilo que não pode ser explicado nem pela tendência e nem pela sazonalidade

In [ ]:
plt.figure(figsize=(12,5))
seasonal.resid.plot()
plt.title("Residual")
plt.xlabel('Date')

A seguir devemos então verificar se a série é estacionária ou não, para isso é utilizaremos o teste do adfuller

O seguinte teste retornará uma tupla de estatísticas do teste do ADF, como Estatística do teste Valor-P; Número de defasagens usadas; Número de observações usadas para a regressão do ADF e um dicionário de Valores críticos.

Resumindo, se o valor-p encontrado pelo teste for < 0,05, a série é estacionária, ja se o valor for acima de 0,05 a série não é estacionária e por esse motivo devemos então normalizala.

Utilizei uma função para simplificar a visualização dos valores, e o valor p encontrado foi superior a 0,05, ou seja, a série não é estacionaria e para realizar as previsões devemos transforma-la em estacionária

In [ ]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(brasil["deaths"])

Para transformar em uma série estacionária, foi necessária a realização de 2 diferenciações. 

In [ ]:
df1 = brasil.copy()
plt.figure(figsize=(12,5))
df1['d2'] = diff(brasil['deaths'],k_diff=2)
df1['d2'][2:].plot();
plt.title("Stacionary timeseries")
plt.ylabel("Date")

In [ ]:
adf_test(df1['d2'])

como podemos ver, agora se tornou uma série estacionária!!

MODELO SARIMA

De uma forma resumida, o ARIMA model é composto por:

AR(p) = Autoregression model, prevemos utilizando a combinação de valores passados da próprima variável. Gera modelos lineares.

Representado pela letra P MA(d) = é o modelo de média móvel. ARMA(p,q) = A junção dos dois acima - representado pela letra Q 

ARIMA(p,q,d) = O mesmo processo que ocorre para o ARMA + aplicação da diferenciação para tornar a série estacionária.

Temos então o SARIMAX (termo genérico), além dos parâmetros (p,q,d) aceita também o (P,D,Q)m, descrevendo os componentes sazonais. P,D e Q representa a regressão sazonal, diferenciação, e média movel, m representa o número de pontos para cada ciclo.

O X representa a variável exógena, como não utilizaremos ela, não entrarei em maiores detalhes.

Para a definição dos melhores parâmetros existe a forma manual e a forma automática, onde a função definirá os melhores parâmetros para nós...

Função: Auto_arima, devo informar ainda o meu dataset, definir os pontos de start do "p" e do "q", definir a sazonalidade e o período de sazonalidade, no caso são de 7 dias.

O objetivo é achar os melhores parâmetros em base no valor AIC, que deve ser o menor possível, com a menor complexidade

Essa ferramenta impede que cometemos erros na hora de analisar os gráficos e realizar transformações nos dados, assim ele nos informa os melhores parâmetros!!



In [ ]:
train_set = brasil[:90]
test_set = brasil[90:]

In [ ]:
sarima = auto_arima(brasil['deaths'],start_p=0, start_q=0,seasonal=True,trace=True, m=7)

In [ ]:
sarima

In [ ]:
sarima.summary()

In [ ]:
start = len(train_set)
end = len(train_set) + len(test_set) - 1

In [ ]:
model = SARIMAX(train_set['deaths'], order=(2, 1, 3),seasonal_order=(0, 1, 1, 7)).fit()

In [ ]:
predictions = model.predict(start,end,typ="levels").rename("SARIMAX(2, 1, 3)x(0, 1, 1, 7)")

In [ ]:
predictions

**Nos gráficos abaixo podemos ver que as predições foram muito próximas aos valores de teste!! O que faremos depois é prever por mais uma semana e depois conferir com os dados**

In [ ]:
test_set['deaths'].plot(label="test set", legend=True)
train_set['deaths'].plot(legend=True, label="train set")
predictions.plot(label="prediction", legend=True)

In [ ]:
test_set['deaths'].plot(label="test set", legend=True)
predictions.plot(label="prediction", legend=True)

In [ ]:
rmse(test_set['deaths'], predictions)

In [ ]:
start = len(train_set)
end = len(train_set) + len(test_set) + 6

In [ ]:
predictions = model.predict(start,end,typ="levels").rename("SARIMAX(2, 1, 3)x(0, 1, 1, 7)")

**Previsões feitas até o dia 05/07/2020, agora vamos conferir com os dados oficiais**

In [ ]:
predictions

In [ ]:
brasil_covid = pd.read_csv("../input/coronavirus/brasil_covid.csv", index_col=1, parse_dates=True,dayfirst=True, sep=";")

In [ ]:
comparacao = brasil_covid[-21:]

In [ ]:
plt.figure(figsize=(8,5))
train_set['deaths'].plot(legend=True, label="Dataset de Treino")
comparacao["obitosAcumulado"].plot(legend=True, label="Dataset de Test")
predictions.plot(legend=True, label="Predictions")
plt.title("Predictions")
plt.ylabel("Mortes")
plt.xlabel("Data")

Novamente os valores foram muito próximos aos reais, chegando ao dia 5 com erro de 7 mortes no total acumulado

In [ ]:
plt.figure(figsize=(8,5))
comparacao["obitosAcumulado"].plot(legend=True, label="Dataset de Test")
predictions.plot(legend=True, label="Predictions")
plt.title("Predictions")
plt.ylabel("Mortes")
plt.xlabel("Data")

In [ ]:
dataframe = pd.DataFrame({"predictions": predictions.values,
                          "mortes":comparacao['obitosAcumulado']})

In [ ]:
dataframe['diferença'] = dataframe['predictions'] - dataframe['mortes']

In [ ]:
dataframe['diferença'] = dataframe['diferença'].astype(int)

In [ ]:
dataframe

**CONCLUSÃO:**

Podemos ver de uma forma simplificada, como a utilização do machine learning e técnicas de Inteligência Artificial, podem contribuir para a soluções de alguns problemas e pode também automatizar tarefas que poderiam levar muito tempo. Podemos concluir que com a utilização de modelos preditivos, é possível prever com uma certa precisão os números de mortes para que as governantes possam tomar atitudes acertivas.

Algumas considerações: 

As previsões foram feitas apenas com relação as datas, mas outros fatores podem inteferir nos números.

Qualquer tipo de erro que eu cometi, podem entrar em contado, sempre gosto de críticas e sugestões

Para a exploração dos textos foi utilizado apenas 500 textos de forma 'aleatória' pois caso utilizasse os 20 mil, tomaria muito tempo de processamento.

Algumas das funções utilizadas tomei como base de outros trabalhos, sendo assim, citei quando necessário.


